In [1]:
import pandas as pd

df = pd.read_excel("all_forks_high_value_combinations.xlsx", index_col="Unnamed: 0")
df

,0,1,2,3,4,5,6,7,8,9,...,36,37,38,39,40,41,Siamese Full TF,Siamese 8020 TF,Twin Full TF,Twin 8020 TF
0,0,0,1,0,1,1,0,0,1,1,...,0,0,0,0,0,0,4.346578,3.667625,4.695021,4.304381
1,0,0,1,0,1,1,0,0,1,1,...,0,0,0,0,0,0,4.510314,3.963938,4.837860,4.120675
2,0,0,1,0,1,1,0,0,1,1,...,0,0,0,0,0,0,4.602205,4.031310,5.278550,4.285867
3,0,0,1,0,1,1,0,0,1,1,...,0,0,0,1,0,0,4.924675,4.098566,5.510090,4.900719
4,0,0,1,0,1,1,0,0,1,1,...,0,0,0,1,0,0,4.935114,4.235371,5.466289,5.268839
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3575,0,0,0,1,1,0,0,0,0,1,...,0,0,1,0,0,0,3.460550,3.493480,5.093630,6.010776
3576,0,0,0,1,1,0,0,0,0,1,...,0,0,1,0,0,0,3.703525,3.622229,5.280568,6.429787
3577,0,0,0,1,1,0,0,0,0,1,...,0,0,1,0,0,0,4.096161,3.727677,5.146357,6.721583
3578,0,0,0,1,1,0,0,0,0,1,...,0,0,1,0,0,0,3.818365,3.650248,4.689723,6.998422


In [2]:
# Models
import torch
import torch.nn.functional as F

class Siamese(torch.nn.Module):
    def __init__(self):
        super(Siamese, self).__init__()
        self.nn1 = torch.nn.Linear(42,30)
        self.nn2 = torch.nn.Linear(30,1)

    def forward(self, x1, x2):
        o1 = self.nn1(x1)
        o1 = F.relu(o1)
        o1 = self.nn2(o1)
            
        o2 = self.nn1(x2)
        o2 = F.relu(o2)
        o2 = self.nn2(o2)
        return torch.sub(o1,o2)
    
class Twin(torch.nn.Module):
    def __init__(self):
        super(Twin, self).__init__()
        self.nn1 = torch.nn.Linear(84,60)
        self.nn2 = torch.nn.Linear(60,30)
        self.nn3 = torch.nn.Linear(30,1) 

            
    def forward(self, x1):
        o1 = self.nn1(x1)
        o1 = F.relu(o1)
        o1 = self.nn2(o1)
        o1 = F.relu(o1)
        o1 = self.nn3(o1)     
        return o1

In [3]:
import pandas as pd
import numpy as np

df_raw = pd.read_excel("produce/ralstonia_high.xlsx", index_col="Samples")

X_right = np.array(df_raw.drop("ralstonia", axis=1).values, dtype=np.float32)
y_right = np.array(df_raw["ralstonia"].values, dtype=np.float32)

In [4]:
# Functions
def get_ralstonia_value(row, X_right, model_arch):
    X_left = np.broadcast_to(np.array(row), (X_right.shape))

    if model_arch == "siamese":
        X_left = torch.tensor(X_left, dtype=torch.float32).to(device)
        X_right = torch.tensor(X_right, dtype=torch.float32).to(device)
        y_pred = model(X_left, X_right)
    elif model_arch == "twin":
        X_pred = torch.tensor(np.concatenate((X_left, X_right), axis=1), dtype=torch.float32).to(device)
        y_pred = model(X_pred)
    
    y_pred = y_pred.cpu().detach().numpy()
    y_pred = y_pred.squeeze()
    y_abs = np.mean(y_pred + y_right)
    return y_abs

In [5]:
# Fork1 Check
model_arch = 'siamese'
device = "cuda"
fork = "fork1"
model = Siamese()
model = torch.load("produce/saved_models/high_fork1.pt").to(device)
ralstonia_values = []
for i in range(df.shape[0]):
    row = df[range(42)].iloc[i].values.tolist()
    val = get_ralstonia_value(row, X_right, model_arch)
    ralstonia_values.append(val)
    
df["Siamese Full Torch"] = ralstonia_values
df

,0,1,2,3,4,5,6,7,8,9,...,37,38,39,40,41,Siamese Full TF,Siamese 8020 TF,Twin Full TF,Twin 8020 TF,Siamese Full Torch
0,0,0,1,0,1,1,0,0,1,1,...,0,0,0,0,0,4.346578,3.667625,4.695021,4.304381,4.081833
1,0,0,1,0,1,1,0,0,1,1,...,0,0,0,0,0,4.510314,3.963938,4.837860,4.120675,4.597339
2,0,0,1,0,1,1,0,0,1,1,...,0,0,0,0,0,4.602205,4.031310,5.278550,4.285867,4.882181
3,0,0,1,0,1,1,0,0,1,1,...,0,0,1,0,0,4.924675,4.098566,5.510090,4.900719,5.131969
4,0,0,1,0,1,1,0,0,1,1,...,0,0,1,0,0,4.935114,4.235371,5.466289,5.268839,5.317682
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3575,0,0,0,1,1,0,0,0,0,1,...,0,1,0,0,0,3.460550,3.493480,5.093630,6.010776,4.737905
3576,0,0,0,1,1,0,0,0,0,1,...,0,1,0,0,0,3.703525,3.622229,5.280568,6.429787,4.997742
3577,0,0,0,1,1,0,0,0,0,1,...,0,1,0,0,0,4.096161,3.727677,5.146357,6.721583,4.819798
3578,0,0,0,1,1,0,0,0,0,1,...,0,1,0,0,0,3.818365,3.650248,4.689723,6.998422,4.903595


In [6]:
# Fork2 Check
model_arch = 'siamese'
device = "cuda"
fork = "fork2"
model = Siamese()
model = torch.load("produce/saved_models/high_fork2.pt").to(device)
ralstonia_values = []
for i in range(df.shape[0]):
    row = df[range(42)].iloc[i].values.tolist()
    val = get_ralstonia_value(row, X_right, model_arch)
    ralstonia_values.append(val)
    
df["Siamese 8020 Torch"] = ralstonia_values
df

,0,1,2,3,4,5,6,7,8,9,...,38,39,40,41,Siamese Full TF,Siamese 8020 TF,Twin Full TF,Twin 8020 TF,Siamese Full Torch,Siamese 8020 Torch
0,0,0,1,0,1,1,0,0,1,1,...,0,0,0,0,4.346578,3.667625,4.695021,4.304381,4.081833,4.587689
1,0,0,1,0,1,1,0,0,1,1,...,0,0,0,0,4.510314,3.963938,4.837860,4.120675,4.597339,4.845356
2,0,0,1,0,1,1,0,0,1,1,...,0,0,0,0,4.602205,4.031310,5.278550,4.285867,4.882181,4.948587
3,0,0,1,0,1,1,0,0,1,1,...,0,1,0,0,4.924675,4.098566,5.510090,4.900719,5.131969,5.199256
4,0,0,1,0,1,1,0,0,1,1,...,0,1,0,0,4.935114,4.235371,5.466289,5.268839,5.317682,5.264114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3575,0,0,0,1,1,0,0,0,0,1,...,1,0,0,0,3.460550,3.493480,5.093630,6.010776,4.737905,2.937263
3576,0,0,0,1,1,0,0,0,0,1,...,1,0,0,0,3.703525,3.622229,5.280568,6.429787,4.997742,3.206933
3577,0,0,0,1,1,0,0,0,0,1,...,1,0,0,0,4.096161,3.727677,5.146357,6.721583,4.819798,3.038770
3578,0,0,0,1,1,0,0,0,0,1,...,1,0,0,0,3.818365,3.650248,4.689723,6.998422,4.903595,3.403852


In [7]:
# Fork2 Check
model_arch = 'twin'
device = "cuda"
fork = "fork3"
model = Siamese()
model = torch.load("produce/saved_models/high_fork3.pt").to(device)
ralstonia_values = []
for i in range(df.shape[0]):
    row = df[range(42)].iloc[i].values.tolist()
    val = get_ralstonia_value(row, X_right, model_arch)
    ralstonia_values.append(val)
    
df["Twin Full Torch"] = ralstonia_values
df

,0,1,2,3,4,5,6,7,8,9,...,39,40,41,Siamese Full TF,Siamese 8020 TF,Twin Full TF,Twin 8020 TF,Siamese Full Torch,Siamese 8020 Torch,Twin Full Torch
0,0,0,1,0,1,1,0,0,1,1,...,0,0,0,4.346578,3.667625,4.695021,4.304381,4.081833,4.587689,5.418316
1,0,0,1,0,1,1,0,0,1,1,...,0,0,0,4.510314,3.963938,4.837860,4.120675,4.597339,4.845356,5.510505
2,0,0,1,0,1,1,0,0,1,1,...,0,0,0,4.602205,4.031310,5.278550,4.285867,4.882181,4.948587,5.810385
3,0,0,1,0,1,1,0,0,1,1,...,1,0,0,4.924675,4.098566,5.510090,4.900719,5.131969,5.199256,6.003075
4,0,0,1,0,1,1,0,0,1,1,...,1,0,0,4.935114,4.235371,5.466289,5.268839,5.317682,5.264114,6.477160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3575,0,0,0,1,1,0,0,0,0,1,...,0,0,0,3.460550,3.493480,5.093630,6.010776,4.737905,2.937263,5.230066
3576,0,0,0,1,1,0,0,0,0,1,...,0,0,0,3.703525,3.622229,5.280568,6.429787,4.997742,3.206933,5.522860
3577,0,0,0,1,1,0,0,0,0,1,...,0,0,0,4.096161,3.727677,5.146357,6.721583,4.819798,3.038770,5.556439
3578,0,0,0,1,1,0,0,0,0,1,...,0,0,0,3.818365,3.650248,4.689723,6.998422,4.903595,3.403852,5.445701


In [8]:
# Fork2 Check
model_arch = 'twin'
device = "cuda"
fork = "fork4"
model = Siamese()
model = torch.load("produce/saved_models/high_fork4.pt").to(device)
ralstonia_values = []
for i in range(df.shape[0]):
    row = df[range(42)].iloc[i].values.tolist()
    val = get_ralstonia_value(row, X_right, model_arch)
    ralstonia_values.append(val)
    
df["Twin 8020 Torch"] = ralstonia_values
df

,0,1,2,3,4,5,6,7,8,9,...,40,41,Siamese Full TF,Siamese 8020 TF,Twin Full TF,Twin 8020 TF,Siamese Full Torch,Siamese 8020 Torch,Twin Full Torch,Twin 8020 Torch
0,0,0,1,0,1,1,0,0,1,1,...,0,0,4.346578,3.667625,4.695021,4.304381,4.081833,4.587689,5.418316,4.928354
1,0,0,1,0,1,1,0,0,1,1,...,0,0,4.510314,3.963938,4.837860,4.120675,4.597339,4.845356,5.510505,4.989141
2,0,0,1,0,1,1,0,0,1,1,...,0,0,4.602205,4.031310,5.278550,4.285867,4.882181,4.948587,5.810385,5.099337
3,0,0,1,0,1,1,0,0,1,1,...,0,0,4.924675,4.098566,5.510090,4.900719,5.131969,5.199256,6.003075,5.287747
4,0,0,1,0,1,1,0,0,1,1,...,0,0,4.935114,4.235371,5.466289,5.268839,5.317682,5.264114,6.477160,5.660887
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3575,0,0,0,1,1,0,0,0,0,1,...,0,0,3.460550,3.493480,5.093630,6.010776,4.737905,2.937263,5.230066,3.125658
3576,0,0,0,1,1,0,0,0,0,1,...,0,0,3.703525,3.622229,5.280568,6.429787,4.997742,3.206933,5.522860,3.460500
3577,0,0,0,1,1,0,0,0,0,1,...,0,0,4.096161,3.727677,5.146357,6.721583,4.819798,3.038770,5.556439,3.240665
3578,0,0,0,1,1,0,0,0,0,1,...,0,0,3.818365,3.650248,4.689723,6.998422,4.903595,3.403852,5.445701,3.520628


In [17]:
df_final = df.drop(42, axis=1)
df_final

,0,1,2,3,4,5,6,7,8,9,...,36,37,38,39,40,41,Siamese Full,Siamese 8020,Twin Full,Twin 8020
0,0,0,1,0,1,1,0,0,1,1,...,0,0,0,0,0,0,4.081833,4.587689,5.418316,4.928354
1,0,0,1,0,1,1,0,0,1,1,...,0,0,0,0,0,0,4.597339,4.845356,5.510505,4.989141
2,0,0,1,0,1,1,0,0,1,1,...,0,0,0,0,0,0,4.882181,4.948587,5.810385,5.099337
3,0,0,1,0,1,1,0,0,1,1,...,0,0,0,1,0,0,5.131969,5.199256,6.003075,5.287747
4,0,0,1,0,1,1,0,0,1,1,...,0,0,0,1,0,0,5.317682,5.264114,6.477160,5.660887
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1871,0,0,1,1,1,0,0,0,1,1,...,0,0,1,0,1,1,4.999185,4.908545,6.494492,6.607216
1872,0,0,1,1,1,0,0,0,1,1,...,0,0,1,0,1,0,5.113395,4.979833,6.777656,6.720832
1873,0,0,1,1,1,0,0,0,1,1,...,0,0,1,0,1,0,4.951515,5.030682,6.693036,6.827909
1874,0,0,1,1,1,1,0,0,1,1,...,0,0,1,0,1,0,4.744020,5.066519,6.601008,6.925135


In [10]:
df.to_excel("tf_and_torch.xlsx")